Spaceship Titanic
=================

https://www.kaggle.com/competitions/spaceship-titanic/code?competitionId=34377

In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer

import warnings
warnings.filterwarnings('ignore')

Set random seed from reproductability
-------------------------------------

In [2]:
def set_seed(seed):
    'Sets the seed of the entire notebook so results are the same every time we run. This is for REPRODUCIBILITY.'
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(42)

Read datasets
-------------

In [3]:
ROOT_PATH = './input_data/'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'

train_data = pd.read_csv(ROOT_PATH+TRAIN_FILE)
test_data = pd.read_csv(ROOT_PATH+TEST_FILE)
print(f'(train) Number of rows = {train_data.shape[0]} and Number of cols = {train_data.shape[1]}')
print(f'(test) Number of rows = {test_data.shape[0]} and Number of cols = {test_data.shape[1]}')

train_data

(train) Number of rows = 8693 and Number of cols = 14
(test) Number of rows = 4277 and Number of cols = 13


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


Analyze train data
------------------

In [4]:
def summary(df):
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    return summ

summary(train_data)

,dtypes,null,unique,min,median,max,mean,std
PassengerId,object,0,8693,0001_01,463001.0,9280_02,inf,NaN
HomePlanet,object,201,3,NaN,NaN,NaN,NaN,NaN
CryoSleep,object,217,2,False,0.0,True,0.358306,0.479531
Cabin,object,199,6560,NaN,NaN,NaN,NaN,NaN
Destination,object,182,3,NaN,NaN,NaN,NaN,NaN
Age,float64,179,80,0.0,27.0,79.0,28.827930,14.489021
VIP,object,203,2,False,0.0,True,0.023439,0.151303
RoomService,float64,181,1273,0.0,0.0,14327.0,224.687617,666.717663
FoodCourt,float64,183,1507,0.0,0.0,29813.0,458.077203,1611.489240
ShoppingMall,float64,208,1115,0.0,0.0,23492.0,173.729169,604.696458


Feature engineering
-------------------

Create a `FunctionTransformer` to remove Name and Cabin (not significative because too many different values), and split the PassengerId in the group id and person id.

In [5]:
def feature_engineering(data):
    return (
            data
            .assign(
                PassengerGGG = [x.split('_')[-0] for x in data['PassengerId']],
                PassengerPP = [x.split('_')[-1] for x in data['PassengerId']],
            )
            .drop(columns=['Name', 'Cabin', 'PassengerId'])
        )

fe_eng = FunctionTransformer(feature_engineering)

fe_eng.fit_transform(train_data)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,PassengerGGG,PassengerPP
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0001,01
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0002,01
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0003,01
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0003,02
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0004,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,9276,01
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,9278,01
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,9279,01
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,9280,01


Create an `Imputer` to fill the missing values with the most frequent value in text columns, and with the mean in numeric ones.

In [6]:
imputer = ColumnTransformer(
    [
        (
            'label_imputer',
            SimpleImputer(strategy='most_frequent'),
            ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'PassengerGGG', 'PassengerPP']
        ),
        (
            'num_imputer',
            SimpleImputer(strategy='mean'),
            ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
        )
    ],
    verbose_feature_names_out=False,
    remainder='passthrough'
).set_output(transform='pandas')

imputer.fit_transform(fe_eng.fit_transform(train_data))

,HomePlanet,CryoSleep,Destination,VIP,PassengerGGG,PassengerPP,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,False,0001,01,39.0,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,TRAPPIST-1e,False,0002,01,24.0,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,TRAPPIST-1e,True,0003,01,58.0,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,TRAPPIST-1e,False,0003,02,33.0,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,TRAPPIST-1e,False,0004,01,16.0,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,True,9276,01,41.0,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,PSO J318.5-22,False,9278,01,18.0,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,TRAPPIST-1e,False,9279,01,26.0,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,55 Cancri e,False,9280,01,32.0,0.0,1049.0,0.0,353.0,3235.0,False


In [7]:
scale_encode = ColumnTransformer(
    [
        (
            'std_scaler',
            StandardScaler(),
            ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
        ),
        (
            'minmax_scaler',
            MinMaxScaler(),
            ['PassengerGGG', 'PassengerPP']
        ),
        (
            'one_hot',
            OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist'),
            ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
        )
    ],
    verbose_feature_names_out=False,
    remainder='passthrough'
).set_output(transform='pandas')

scale_encode.fit_transform(imputer.fit_transform(fe_eng.fit_transform(train_data)))

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGGG,PassengerPP,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Transported
0,0.709437,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,False
1,-0.336717,-0.175364,-0.281669,-0.248968,0.211505,-0.230194,0.000108,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,True
2,2.034566,-0.275409,1.955616,-0.290817,5.694289,-0.225782,0.000216,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,False
3,0.290975,-0.340590,0.517406,0.330225,2.683471,-0.098708,0.000216,0.142857,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,False
4,-0.894666,0.118709,-0.243409,-0.038048,0.225732,-0.267258,0.000323,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.848924,-0.340590,3.989682,-0.290817,1.184286,-0.203720,0.999569,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,False
8689,-0.755179,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.999784,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,False
8690,-0.197230,-0.340590,-0.287314,2.842851,-0.275774,-0.269023,0.999892,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,True
8691,0.221232,-0.340590,0.370637,-0.290817,0.037223,2.585740,1.000000,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,False


Combine all transformers in a pipeline.

In [8]:
preproc = Pipeline(
    [
        ('fe_eng', fe_eng),
        ('imputer', imputer),
        ('scale_encode', scale_encode)
    ]
).set_output(transform='pandas')

preproc.fit_transform(train_data)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGGG,PassengerPP,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True,Transported
0,0.709437,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,False
1,-0.336717,-0.175364,-0.281669,-0.248968,0.211505,-0.230194,0.000108,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,True
2,2.034566,-0.275409,1.955616,-0.290817,5.694289,-0.225782,0.000216,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,False
3,0.290975,-0.340590,0.517406,0.330225,2.683471,-0.098708,0.000216,0.142857,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,False
4,-0.894666,0.118709,-0.243409,-0.038048,0.225732,-0.267258,0.000323,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.848924,-0.340590,3.989682,-0.290817,1.184286,-0.203720,0.999569,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,False
8689,-0.755179,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.999784,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,False
8690,-0.197230,-0.340590,-0.287314,2.842851,-0.275774,-0.269023,0.999892,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,True
8691,0.221232,-0.340590,0.370637,-0.290817,0.037223,2.585740,1.000000,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,False


Create a separated transformer to drop the target column

In [9]:
def drp_trg(data):
    if hasattr(data, 'Transported'):
        return data.drop(columns=['Transported'])
    else:
        return data

drop_target = FunctionTransformer(drp_trg)